# **Restricted Bolzman machine**

In [1]:
#Importing the libraries
import pickle

from pathlib import Path

import pandas as pd
import torch.utils.data
from sklearn.model_selection import train_test_split

from models.rbm import RBM
from read_data import read_data, create_tabular_data


In [2]:
users = pd.read_csv("ml-1m/users.dat", sep="::", header=None, engine="python", encoding="latin-1")
ratings, movies = read_data(Path('ml-1m'))

In [3]:
nb_users = len(users)
nb_movies = len(movies)

In [4]:
nb_movies, nb_users

(3883, 6040)

In [5]:
df = create_tabular_data(ratings, users, movies)
train_set, test_set = train_test_split(df, test_size=0.2)

In [6]:
train_set = torch.FloatTensor(train_set.to_numpy())
test_set = torch.FloatTensor(test_set.to_numpy())

In [7]:
# Converting the ratings into binary ratings 1(liked) and 0(disliked)
train_set[train_set==0] = -1
train_set[train_set==1] = 0
train_set[train_set==2] = 0
train_set[train_set>=3] = 1
test_set[test_set==0] = -1
test_set[test_set==1] = 0
test_set[test_set==2] = 0
test_set[test_set>=3] = 1


In [8]:
nv = len(train_set[0])
nh = 100
nb_epochs = 10
batch_size = 500

In [9]:
rbm = RBM(nv,nh)
rbm.train_model(nb_epochs, batch_size, train_set)

epoch: 1 loss: tensor(0.2581)
epoch: 2 loss: tensor(0.2233)
epoch: 3 loss: tensor(0.2296)
epoch: 4 loss: tensor(0.2304)
epoch: 5 loss: tensor(0.2303)
epoch: 6 loss: tensor(0.2309)
epoch: 7 loss: tensor(0.2315)
epoch: 8 loss: tensor(0.2298)
epoch: 9 loss: tensor(0.2310)
epoch: 10 loss: tensor(0.2314)


In [10]:
import copy
best_weights = copy.deepcopy(rbm.state_dict())
with open('RBM.weights', 'wb') as file:
    pickle.dump(best_weights, file)

In [11]:
rbm.test(train_set, test_set)

 loss: tensor(0.1977)


In [12]:
rec = rbm.predict_recommendations(test_set)
rec[:50]

predicting recommendations for user: 92


C:\Users\murtu\PycharmProjects\Recommendation-System-platform\models\rbm.py:85: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rec_df = out_df[out_df['ground_truth'] == -1][out_df['predicted_ratings'] == 1].sort_values(


[2619, 2609, 2616, 2, 2613]